<a href="https://colab.research.google.com/github/marjg/datasets2/blob/main/Barbara_Margie_Konrad_TF_Regression_Miami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/marjg/datasets3/main/WineQT.csv')
df.sample(5)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
311,15.6,0.685,0.76,3.70,0.100,6.0,43.0,1.00320,2.95,0.68,11.2,7,442
268,8.3,0.260,0.42,2.00,0.080,11.0,27.0,0.99740,3.21,0.80,9.4,6,380
1043,7.3,0.480,0.32,2.10,0.062,31.0,54.0,0.99728,3.30,0.65,10.0,7,1466
383,11.2,0.500,0.74,5.15,0.100,5.0,17.0,0.99960,3.22,0.62,11.2,5,539
733,6.9,0.440,0.00,1.40,0.070,32.0,38.0,0.99438,3.32,0.58,11.4,6,1045


In [ ]:
df.isnull().sum()

In [ ]:
# Creating a new column 'is_good' based on the 'quality' column
# df['is_good'] = (df['quality'] >= 6).astype(int)

# Display the updated dataframe with the new column
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [ ]:
df.sample

In [ ]:
to_drop = ['Id']
df = df.drop(to_drop, axis=1)

In [ ]:
# Ensure 'quality' is of type float32
df['quality'] = df['quality'].astype('float32')


In [ ]:
df.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                 float32
dtype: object

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()

X = df.drop('quality', axis=1)
y = df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
X_test[0]

In [ ]:
y[0]

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# df['quality'] = df['quality'].astype('float32')

# Custom loss function
def rmse(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is cast to float32
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(loss=rmse, optimizer=Adam(), metrics=[rmse])

model.fit(X_train, y_train, epochs=25, batch_size=32)


Epoch 1/25
29/29 [==============================] - 1s 2ms/step - loss: 5.0084 - rmse: 4.9870
Epoch 2/25
29/29 [==============================] - 0s 2ms/step - loss: 1.5569 - rmse: 1.5510
Epoch 3/25
29/29 [==============================] - 0s 2ms/step - loss: 0.9593 - rmse: 0.9590
Epoch 4/25
29/29 [==============================] - 0s 2ms/step - loss: 0.8436 - rmse: 0.8405
Epoch 5/25
29/29 [==============================] - 0s 2ms/step - loss: 0.8037 - rmse: 0.8055
Epoch 6/25
29/29 [==============================] - 0s 2ms/step - loss: 0.7762 - rmse: 0.7778
Epoch 7/25
29/29 [==============================] - 0s 2ms/step - loss: 0.7629 - rmse: 0.7634
Epoch 8/25
29/29 [==============================] - 0s 2ms/step - loss: 0.7447 - rmse: 0.7488
Epoch 9/25
29/29 [==============================] - 0s 2ms/step - loss: 0.7325 - rmse: 0.7358
Epoch 10/25
29/29 [==============================] - 0s 2ms/step - loss: 0.7114 - rmse: 0.7111
Epoch 11/25
29/29 [==============================] - 0s 2ms

In [ ]:
# Generate predictions using the trained model
predictions = model.predict(X_test)
# Check the shapes of y_test and predictions
print("y_test shape:", y_test.shape)
print("predictions shape:", predictions.shape)

# Flatten both y_test and predictions if necessary
if len(y_test.shape) > 1:
    y_test = y_test.flatten()

if len(predictions.shape) > 1:
    predictions = predictions.flatten()

# Calculate RMSE
rmse_value = rmse(y_test, predictions)
print(f'RMSE on the test set: {rmse_value.numpy()}')


8/8 [==============================] - 0s 2ms/step
y_test shape: (229,)
predictions shape: (229, 1)
RMSE on the test set: 0.6216378808021545


In [ ]:
# Calculate mean absolute error (MAE)
mae = tf.keras.losses.MeanAbsoluteError()
mae_value = mae(y_test, predictions).numpy()

# Print RMSE and MAE
print(f"RMSE on the test set: {rmse_value:.2f}")
print(f"Mean Absolute Error (MAE) on the test set: {mae_value:.2f}")


RMSE on the test set: 0.62
Mean Absolute Error (MAE) on the test set: 0.47


In [ ]:
# Create a new data point with 14 features
# You can copy from the last dataset and remove the sales price
# LATITUDE	LONGITUDE	LND_SQFOOT	TOT_LVG_AREA	SPEC_FEAT_VAL	RAIL_DIST	CNTR_DIST	SUBCNTR_DI	HWY_DIST	age	avno60plus	month_sold	structure_quality	dist_to_water
# 15.6	0.685	0.76	3.70	0.100	6.0	43.0	1.00320	2.95	0.68	11.2
new_data = np.array([[15.6,0.685,0.76,3.70,0.100,6.0,43.0,1.00320,2.95,0.68,11.2]])

# Preprocess the new data point
scaled_new_data = scaler.transform(new_data)

# Make a prediction using the preprocessed new data point
new_prediction = model.predict(scaled_new_data)

# Print the prediction
print(f"Predicted value for the new data point: {new_prediction[0][0]:.2f}")


1/1 [==============================] - 0s 20ms/step
Predicted value for the new data point: 5.91


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load dataset
url = 'https://raw.githubusercontent.com/marjg/datasets3/main/WineQT.csv'
data = pd.read_csv(url)

# Split the data into features and target label
X = data.drop(columns=['quality', 'Id'])
y = data['quality']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a Sequential model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# Optional: Predictions and further analysis
# predictions = model.predict(X_test)


Epoch 1/100
23/23 [==============================] - 2s 25ms/step - loss: 20.2957 - val_loss: 8.1305
Epoch 2/100
23/23 [==============================] - 0s 6ms/step - loss: 4.5244 - val_loss: 3.7987
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 2.4071 - val_loss: 2.6141
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 1.7520 - val_loss: 2.2970
Epoch 5/100
23/23 [==============================] - 0s 3ms/step - loss: 1.4937 - val_loss: 2.1371
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 1.3405 - val_loss: 2.0236
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 1.2056 - val_loss: 1.7935
Epoch 8/100
23/23 [==============================] - 0s 4ms/step - loss: 1.1144 - val_loss: 1.6907
Epoch 9/100
23/23 [==============================] - 0s 3ms/step - loss: 1.0209 - val_loss: 1.5599
Epoch 10/100
23/23 [==============================] - 0s 3ms/step - loss: 0.9290 - val_loss: 1.4100
Epoch 1